In [1]:
#This one uses an active scenario in the GUI


from veneer.manage import start, create_command_line, kill_all_now
import veneer
import pandas as pd
import gc
import numpy as np
#from veneer.pest_runtime import *


veneer_port = 9876
v = veneer.Veneer(veneer_port)

#nodeOfInterestName = '120212A'
#outputCSV = 'D:/aa/testingSubCats.csv'
#outputCSV = r'C:\DDrive\WetTropics\Johnstone\contribCatchs.csv'

In [2]:
#Check we have the right scenario
sDate = v.model.get('scenario.CurrentConfiguration.StartDate')
print(sDate)

eDate = v.model.get('scenario.CurrentConfiguration.EndDate')
print(eDate)

#v.run_model()

1/07/1986 12:00:00 AM
30/06/2014 12:00:00 AM


In [3]:
the_network = v.network()
#outlets = the_network.outlet_nodes()
#outlets

In [4]:
def processLinks(theNetwork, currentLink):
    theLinkName = currentLink['properties']['name']
    theLinkID = currentLink['id']
    
    #print("Now processing: " + theLinkName)
          
    # I think this will actually return a catchment record, not just an ID
    theCat = theNetwork['features'].find_by_link(theLinkID)[0]
    
    #print(theCat)
    
    thisCatName = theCat['properties']['name']
    #print(thisCatName)
    #thisCatArea = theCat['properties']['areaInSquareMeters']
    
    GlobalCatchList.append(thisCatName)
    
    #print("Processing " + thisCatName + " with area of " + str(thisCatArea))
        
    fNodeID = currentLink['properties']['from_node']
    
    #print("Node info: " + fNodeID)
    
    fNode = the_network['features'].find_by_id(fNodeID)
    #print(fNode)
    
    fNodeName = fNode[0]['properties']['name']
    #print(fNodeName)
    
    ## recurse
    #print(the_network.upstream_links(fNode[0]))
    for upLink in the_network.upstream_links(fNode[0]):
        #tNodeID = upLink['properties']['to_node']
        #tNode = the_network['features'].find_by_id(tNodeID)
        #print("Now processing: " + upLink['properties']['name'] + " upstream of " + tNode[0]['properties']['name'] + " with initial Region: " + currentReg)
        processLinks(theNetwork, upLink)

    return

In [14]:
GlobalCatchList = []

elementOfInterestName = 'link for catchment SC #325'
outputCSV = r'E:\BurnettMaryRC11\contribCatchs_UpperBurnett.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Added our starting link first
Done exporting


In [15]:
GlobalCatchList = []

elementOfInterestName = 'link for catchment SC #107'
outputCSV = r'E:\BurnettMaryRC11\contribCatchs_BarkerBarambah.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Added our starting link first
Done exporting


In [16]:
GlobalCatchList = []

elementOfInterestName = 'link for catchment SC #244'
outputCSV = r'E:\BurnettMaryRC11\contribCatchs_AuburnBoyne1.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Added our starting link first
Done exporting


In [17]:
GlobalCatchList = []

elementOfInterestName = 'link for catchment SC #178'
outputCSV = r'E:\BurnettMaryRC11\contribCatchs_AuburnBoyne2.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Added our starting link first
Done exporting


In [18]:
GlobalCatchList = []

elementOfInterestName = 'link for catchment SC #501'
outputCSV = r'E:\BurnettMaryRC11\contribCatchs_UpperMary.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Added our starting link first
Done exporting


In [19]:
GlobalCatchList = []

elementOfInterestName = 'Outlet Node18'
outputCSV = r'E:\BurnettMaryRC11\contribCatchs_NorthBurrum.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Done exporting


In [20]:
GlobalCatchList = []

elementOfInterestName = 'Outlet Node19'
outputCSV = r'E:\BurnettMaryRC11\contribCatchs_SouthBurrum.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Done exporting


In [21]:
GlobalCatchList = []

elementOfInterestName = 'Outlet Node16'
outputCSV = r'E:\BurnettMaryRC11\contribCatchs_SouthElliott.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Done exporting


In [22]:
GlobalCatchList = []

elementOfInterestName = 'Outlet Node14'
outputCSV = r'E:\BurnettMaryRC11\contribCatchs_NorthElliott.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Done exporting


In [23]:
GlobalCatchList = []

elementOfInterestName = 'Outlet Node20'
outputCSV = r'E:\BurnettMaryRC11\contribCatchs_HerveyBay.csv'

elementOfInterest = the_network['features'].find_by_name(elementOfInterestName)

#print(elementOfInterest[0]['properties']['feature_type'])

if elementOfInterest[0]['properties']['feature_type'] == 'link':
    #Need to add the link/catchent to our collection
    theLinkID = elementOfInterest[0]['id']
    theCat = the_network['features'].find_by_link(theLinkID)[0]
    thisCatName = theCat['properties']['name']
    GlobalCatchList.append(thisCatName)
    print("Added our starting link first")

#print("NOF: " + nodeOfInterest[0]['properties']['name'])
## now we begin processing
for upLink in the_network.upstream_links(elementOfInterest[0]):
    #print("Recursing for link: " + upLink['properties']['name'])
    processLinks(the_network, upLink)

contCatchsDF = pd.DataFrame(GlobalCatchList, columns=['SubCats'])
#contCatchsDF
contCatchsDF.to_csv(outputCSV, index=False)
print("Done exporting")

Done exporting
